In [ ]:
import subprocess
import calendar
import pickle
import numpy as np

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
dates = []
max_jobs = 32*10
for year in range(2016,2024): #  2019
    for month in range(1,13): # 1,1
        _, num_days = calendar.monthrange(year, month)
        for day in range(1,num_days + 1): # +1 for the range bound of [ )
            # print(f'{year}-{month}-{day}')
            dates.append((year, month, day))

date_split = np.array_split(dates, max_jobs)

date_split = [arr for arr in date_split if arr.size > 0]



In [ ]:
for chunk in range(len(date_split)):
    with open(f"chunk_{chunk}.pkl", 'wb') as f:
        pickle.dump(date_split[chunk], f)


    job = fr"""
#!/bin/bash
#PBS -S /bin/bash
#PBS -j oe
#PBS -M christopher.marsh@ec.gc.ca
#PBS -m abe
#PBS -q development
#PBS -l walltime=06:00:00
#PBS -l select=1:ncpus=1:mem=64gb

. ssmuse-sh -p /fs/ssm/eccc/cmd/cmds/env/python/cmds_python_latest

cd /home/chm003/project/hrdps
python eccc-extract-hrdps/main.py chunk_{chunk}.pkl
rm chunk_{chunk}.pkl
"""
    
    # print(f'submit {chunk}')
    subprocess.run(['/opt/pbs/bin/qsub'], input=job, text=True)



In [ ]:
import xarray as xr

In [ ]:
ds = xr.open_mfdataset('20170102.nc')
ds.isel(time=1).RH.plot()